## Modèle


In [102]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np
import pandas as pd
import tensorflow as tf


In [103]:
X = pd.read_pickle('datasets/X_all.pkl')

In [104]:
X.shape

(542, 132)

In [105]:
import json, pickle

with open("datasets/labels_mapping.json", "r", encoding="utf-8") as f:
    mp = json.load(f)

print("labels ordre modèle:", mp["labels"])
print("id2label:", mp["id2label"])  # clés en str


with open("datasets/y_all.pkl", "rb") as f:
    y = pickle.load(f)




labels ordre modèle: ['cobra', 'tree', 'downdog', 'forwardfold', 'chair', 'warrior2', 'warrior3', 'plank', 'lotus']
id2label: {'0': 'cobra', '1': 'tree', '2': 'downdog', '3': 'forwardfold', '4': 'chair', '5': 'warrior2', '6': 'warrior3', '7': 'plank', '8': 'lotus'}


In [106]:
id2label = {int(k): v for k, v in mp["id2label"].items()}
label2id = mp.get("label2id", None)  # celui-ci a des clés string (= labels) donc OK

id2label

{0: 'cobra',
 1: 'tree',
 2: 'downdog',
 3: 'forwardfold',
 4: 'chair',
 5: 'warrior2',
 6: 'warrior3',
 7: 'plank',
 8: 'lotus'}

In [107]:
y.shape

(542,)

In [108]:
y = y.reshape((y.shape[0],1))

In [109]:
y.shape

(542, 1)

## Model split


In [110]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [111]:
X_train.shape

(363, 132)

In [112]:
X_test.shape

(179, 132)

In [113]:
y_test.shape, y_train.shape


((179, 1), (363, 1))

## Model trained

In [140]:
model = Sequential([
    Dense(254, activation="relu", input_shape=(132,)),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dense(10, activation="softmax") ## 10 classes
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [141]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",     # ce qu'on surveille
    patience=20,             # nb d'époques sans amélioration
    restore_best_weights=True
)

model.fit(
    X_train,
    y_train,
    epochs=500,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop])

Epoch 1/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.1460 - loss: 2.2468 - val_accuracy: 0.2235 - val_loss: 2.0706
Epoch 2/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2452 - loss: 2.0654 - val_accuracy: 0.4749 - val_loss: 1.8390
Epoch 3/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3388 - loss: 1.8398 - val_accuracy: 0.5251 - val_loss: 1.5716
Epoch 4/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4160 - loss: 1.6275 - val_accuracy: 0.6536 - val_loss: 1.3738
Epoch 5/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5014 - loss: 1.4213 - val_accuracy: 0.5978 - val_loss: 1.1400
Epoch 6/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5565 - loss: 1.2760 - val_accuracy: 0.7598 - val_loss: 0.9709
Epoch 7/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6033 - loss: 1.1161 - val_accuracy: 0.7877 - val_loss: 0.8396
Epoch 8/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6501 - loss: 0.9952 - val_accuracy: 0.7

In [142]:
pred = model.predict(X_test)
pose_id = np.argmax(pred, axis=1)



6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [143]:
pose_id

array([5, 0, 5, 1, 3, 1, 2, 8, 8, 8, 0, 2, 6, 0, 8, 3, 7, 1, 8, 0, 2, 5,
       1, 5, 8, 0, 1, 6, 0, 2, 2, 2, 6, 3, 8, 8, 7, 0, 4, 5, 7, 3, 2, 2,
       2, 1, 2, 2, 4, 0, 5, 8, 8, 0, 8, 2, 1, 6, 3, 4, 3, 4, 0, 8, 0, 8,
       7, 5, 4, 5, 1, 2, 3, 2, 0, 4, 0, 2, 6, 1, 3, 0, 1, 5, 0, 8, 8, 0,
       7, 1, 1, 7, 6, 6, 1, 1, 2, 0, 3, 2, 8, 2, 0, 8, 2, 0, 1, 8, 8, 4,
       2, 8, 5, 4, 5, 6, 4, 1, 3, 8, 2, 8, 2, 5, 0, 5, 4, 5, 6, 7, 8, 1,
       5, 3, 2, 7, 5, 7, 7, 2, 6, 6, 0, 5, 7, 4, 7, 5, 7, 7, 0, 7, 0, 4,
       4, 2, 4, 2, 1, 8, 8, 0, 1, 6, 3, 6, 0, 2, 0, 0, 8, 0, 2, 3, 3, 2,
       1, 2, 6])

In [144]:
pred


array([[1.06702664e-07, 3.04429355e-04, 2.82738472e-10, ...,
        2.14697931e-08, 8.64445315e-09, 2.97582875e-10],
       [9.97182965e-01, 1.62399061e-07, 4.26969609e-06, ...,
        2.74508144e-03, 3.11473450e-05, 6.67125306e-08],
       [2.38733513e-08, 6.47679772e-05, 1.14178812e-10, ...,
        1.32989051e-08, 8.24766810e-10, 9.47328674e-11],
       ...,
       [1.60280445e-06, 9.95685458e-01, 5.99650150e-07, ...,
        1.06050902e-06, 3.19197607e-05, 2.11367102e-08],
       [4.44552645e-10, 4.02756062e-13, 9.99933243e-01, ...,
        5.28175406e-05, 3.30198833e-15, 9.32308918e-12],
       [4.95626637e-07, 3.78587650e-08, 1.17133322e-06, ...,
        1.65728095e-04, 8.03929284e-11, 2.30547510e-08]], dtype=float32)

In [145]:
X_test[0:1].shape

(1, 132)

## Model Test 

In [146]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Évaluation Keras
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy (keras): {acc:.4f}")


Test loss: 0.1330
Test accuracy (keras): 0.9609


In [147]:
# Accuracy "sklearn" + matrice de confusion (plus explicite)
y_proba = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_proba, axis=1)
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))


Confusion matrix:
 [[28  0  0  0  0  0  0  1  2]
 [ 0 20  0  0  0  0  0  0  0]
 [ 0  0 30  0  0  0  0  0  0]
 [ 0  0  0 13  0  0  0  0  0]
 [ 0  0  0  0 14  0  1  1  0]
 [ 0  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  0  0 13  0  0]
 [ 0  0  0  1  0  1  0 13  0]
 [ 0  0  0  0  0  0  0  0 24]]


In [148]:
print("\nClassification report:\n", classification_report(y_test, y_pred, target_names=[id2label[i] for i in sorted(id2label)]))


Classification report:
               precision    recall  f1-score   support

       cobra       1.00      0.90      0.95        31
        tree       1.00      1.00      1.00        20
     downdog       1.00      1.00      1.00        30
 forwardfold       0.93      1.00      0.96        13
       chair       1.00      0.88      0.93        16
    warrior2       0.94      1.00      0.97        17
    warrior3       0.93      1.00      0.96        13
       plank       0.87      0.87      0.87        15
       lotus       0.92      1.00      0.96        24

    accuracy                           0.96       179
   macro avg       0.95      0.96      0.96       179
weighted avg       0.96      0.96      0.96       179



## Prediction du modèle avec une photo

In [149]:
import cv2
import os
import mediapipe as mp
from mediapipe import solutions
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [150]:

data_root = r'C:\Users\mvana\Documents\Formation data scientist\20. ACV\Posture_yoga\data'
img_path = os.path.join(data_root, 'downdog', 'downdog (24).jpg')


In [151]:


def get_landmarks(image_rgb, pose_model): 
    '''
    Récupère les landmarks d'une image donnée en RGB via un modèle déjà chargé.
    Return: un numpy array (33,4) ou None si rien n'est trouvé.
    '''

    results = pose_model.process(image_rgb)
    
    if results.pose_landmarks:

        pose_np = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark])
        return pose_np

    return None

In [152]:
arbre = cv2.imread(img_path)

print("Image chargée:", arbre is not None)
image_rgb = cv2.cvtColor(arbre, cv2.COLOR_BGR2RGB)

Image chargée: True


In [153]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_model:

    landmarks = get_landmarks(image_rgb,pose_model)

In [154]:
landmarks_flatten = landmarks.flatten().reshape(1,-1)

In [155]:
results = model.predict(landmarks_flatten)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [156]:
np.argmax(results)

np.int64(3)

Optin facilité pour récupérer facilement une image et la prédire vite 

In [157]:
import os
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose

def predict_pose_from_path(img_path, model, id2label, min_det=0.5, min_track=0.5):
    """
    Prédit la pose à partir d'un chemin d'image.
    Retour: (label, proba, pred_id, probs)
    """
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        raise FileNotFoundError(f"Impossible de lire l'image: {img_path}")

    image_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    with mp_pose.Pose(min_detection_confidence=min_det, min_tracking_confidence=min_track) as pose_model:
        landmarks = get_landmarks(image_rgb, pose_model)

    if landmarks is None:
        return "no_pose_detected", 0.0, None, None

    X = landmarks.flatten().reshape(1, -1).astype(np.float32)  # (1,132)

    probs = model.predict(X, verbose=0)[0]   # (num_classes,)
    pred_id = int(np.argmax(probs))
    label = id2label[pred_id]
    proba = float(probs[pred_id])

    return label, proba, pred_id, probs


In [158]:
import os
import random

def predict_on_folder_random(folder_path, model, id2label, n=10, seed=None):
    files = [f for f in os.listdir(folder_path)
             if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    if len(files) == 0:
        print("Aucune image trouvée.")
        return

    if seed is not None:
        random.seed(seed)

    # Tire n fichiers distincts au hasard (ou tous si n > nb images)
    chosen = random.sample(files, k=min(n, len(files)))

    for f in chosen:
        p = os.path.join(folder_path, f)
        label, proba, pred_id, _ = predict_pose_from_path(p, model, id2label)
        print(f"{f:30s} -> {label:12s}  ({proba:.3f})")



In [159]:
data_root = r'C:\Users\mvana\Documents\Formation data scientist\20. ACV\Posture_yoga\data'
img_path = os.path.join(data_root, 'downdog', 'downdog (19).jpg')

label, proba, pred_id, probs = predict_pose_from_path(img_path, model, id2label)
print("➡️", label, f"(proba={proba:.3f}, id={pred_id})")

➡️ forwardfold (proba=0.983, id=3)


In [160]:
data_root = r'C:\Users\mvana\Documents\Formation data scientist\20. ACV\Posture_yoga\data'
img_path = os.path.join(data_root, 'child', 'File27.png')

label, proba, pred_id, probs = predict_pose_from_path(img_path, model, id2label)
print("➡️", label, f"(proba={proba:.3f}, id={pred_id})")

➡️ forwardfold (proba=0.982, id=3)


In [161]:
import os

def test_all_classes(data_root, model, id2label, poses, n_per_class=5, seed=None):
    """
    Pour chaque pose (dossier), tire n_per_class images au hasard et affiche la prédiction.
    """
    for pose in poses:
        folder = os.path.join(data_root, pose)
        print(f"\n=== {pose.upper()} ===")

        if not os.path.isdir(folder):
            print(f"[WARN] Dossier introuvable: {folder}")
            continue

        predict_on_folder_random(folder, model, id2label, n=n_per_class, seed=seed)

# Exemple d'utilisation
poses = ["cobra", "tree", "downdog", "forwardfold", "chair",
         "warrior2", "warrior3", "plank", "lotus"]

test_all_classes(data_root, model, id2label, poses, n_per_class=5, seed=None)



=== COBRA ===
3_322.jpg                      -> cobra         (0.551)
1_152.jpg                      -> cobra         (0.981)
File33.png                     -> cobra         (0.993)
images35.jpg                   -> cobra         (0.994)
PXL_20251217_103534696.MP.jpg  -> cobra         (0.926)

=== TREE ===
97.jpg                         -> tree          (0.999)
images133.jpg                  -> warrior2      (0.562)
00000124.jpg                   -> tree          (0.996)
00000007 (1).jpg               -> tree          (0.995)
152.jpg                        -> tree          (0.993)

=== DOWNDOG ===
downdog (80).jpg               -> forwardfold   (0.830)
downdog (51).jpg               -> downdog       (1.000)
downdog (20).jpg               -> forwardfold   (0.836)
downdog (6).jpg                -> downdog       (1.000)
downdog (21).png               -> downdog       (0.885)

=== FORWARDFOLD ===
File1.png                      -> forwardfold   (0.996)
File55.png                     -> no_

## Save the model in pickle

In [162]:
model.save('yoga_model_9pose.keras')

In [165]:
import json
import os

# Crée la liste labels dans l'ordre des ids (TRÈS important)
labels = [id2label[i] for i in sorted(id2label.keys())]


config = {
    "model_path": "yoga_model_9pose.keras", 
    "num_classes": len(labels),
    "labels": labels,                       # ordre = indices de sortie du modèle
    "id2label": {str(k): v for k, v in id2label.items()},   # JSON => clés string
}

config_path = os.path.join("yoga_config.json")

with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print("✅ Config écrite:", config_path)
print("labels:", labels)


✅ Config écrite: yoga_config.json
labels: ['cobra', 'tree', 'downdog', 'forwardfold', 'chair', 'warrior2', 'warrior3', 'plank', 'lotus']


In [164]:
config

{'model_path': 'yoga_model_10pose.keras',
 'num_classes': 9,
 'labels': ['cobra',
  'tree',
  'downdog',
  'forwardfold',
  'chair',
  'warrior2',
  'warrior3',
  'plank',
  'lotus'],
 'id2label': {'0': 'cobra',
  '1': 'tree',
  '2': 'downdog',
  '3': 'forwardfold',
  '4': 'chair',
  '5': 'warrior2',
  '6': 'warrior3',
  '7': 'plank',
  '8': 'lotus'}}